In [ ]:
import yfinance as yf
import pandas as pd
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_last_dividend(ticker, *args):
  '''
  Params:
  ticker = Ticker or list of tickers whose stock you want the information.
  ============================================================================
  Output:
  ticker = Returns a DataFrame with last dividend date and value for each ticker.
  '''
  tickers = (ticker,) + tuple(args)
  last_dividends = pd.DataFrame()
  for ticker in tickers:
    if type(ticker) != str and type(ticker) != list:
      raise TypeError("Ticker should be passed as a string or a list of strings")
    if type(ticker) == list and len(ticker)>0:
      ticker = [ticker[i] for i in range(len(ticker)) if type(ticker[i]) == str]
      for n in range(len(ticker)):
        ticker_value = ticker[n]
        ticker_obj = yf.Ticker(ticker_value)
        dividends = ticker_obj.dividends.reset_index().tail(1)
        dividends['Date'] = dividends["Date"].astype(str).apply(lambda x: x.split(' ').pop(0))
        dividends['Ticker'] = ticker_value
        last_dividends = pd.concat([last_dividends, dividends], ignore_index=True, axis=0)
    else:
      if type(ticker) != str: raise TypeError("Ticker should be passed as a string, multiple strings or a list of strings")
      ticker_value = ticker
      ticker_obj = yf.Ticker(ticker_value)
      dividends = ticker_obj.dividends.reset_index().tail(1)
      dividends['Date'] = dividends["Date"].astype(str).apply(lambda x: x.split(' ').pop(0))
      dividends['Ticker'] = ticker_value
      last_dividends = pd.concat([last_dividends, dividends], ignore_index=True, axis=0)
  return last_dividends

In [12]:
get_last_dividend(("XPML11.SA"), ["KNIP11.SA", 2], ("PETR4.SA", 2, 5))

,Date,Dividends,Ticker
0,2025-02-19,0.920000,XPML11.SA
1,2025-02-03,0.950000,KNIP11.SA
2,2024-12-26,1.339555,PETR4.SA


In [7]:
dvds = get_last_dividend(("XPML11.SA"), ["KNIP11.SA", 2], ("PETR4.SA", 2, 5))
dvds.to_excel('./data/teste.xlsx', index=False)

In [ ]:

workbook = load_workbook('data/teste.xlsx')  # Replace with your actual file name
sheet = workbook.active  # Assuming you want to append to the active sheet
# Assuming 'dvds' is your DataFrame with the last dividends
dvds = get_last_dividend(("XPLG11.SA"))
for index, row in dds.iterrows():
    sheet.append([row['Date'], row['Dividends'], row['Ticker']])  # Adjust column names as needed

workbook.save('data/teste.xlsx')  # Save the changes back to the file